In [ ]:
from ratdata import data_manager as dm, process, ingest
import scipy.signal as signal
import pathlib

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
recording_files = dm.RecordingFile.select()

In [ ]:
recalculate = False
scale = 1
beta_low = 13
beta_high = 30
oof_fit_low = 2
oof_fit_high = 100
total_low = 10
total_high = 100

for i, f in enumerate(recording_files):
    if recalculate or (f.slice.count() == 1 and f.slice.get().updated):
        fname = pathlib.Path(f.dirname) / f.filename
        recording = ingest.read_mce_matlab_file(str(fname))
        fs = int(1 / recording.dt)
        if f.slice.count() == 1:
            slice = f.slice.get()
            data, tt = process.trim_recording(recording.electrode_data.mean(0), fs, slice.start, slice.length)
        else:
            data = recording.electrode_data.mean(0)
        recording_beta_power = process.power_in_frequency_band(data, beta_low, beta_high, fs)
        recording_total_power = process.power_in_frequency_band(data, total_low, total_high, fs)
        ff, pxx = signal.welch(data, fs, nperseg=fs)
        m, b = process.fit_oof(ff, pxx, oof_fit_low, oof_fit_high)
        rp_id = dm.upsert_power_record(f, recording_beta_power, recording_total_power, m, b)
        print('[%d/%d] %s: %d' % (i + 1, len(recording_files), f.filename, rp_id))
    